# GET DATA

In [1]:
import pandas as pd
import pymysql
import json
import traceback
from sqlalchemy import create_engine

# ==============================================================================
# KONFIGURASI DATABASE - MOHON SESUAIKAN BAGIAN INI
# ==============================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel untuk masing-masing fungsi
# PRICE_DATA_TABLE = 'data_saham_max_all_mei'
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'


def get_analysis_results(ticker_symbol: str) -> pd.DataFrame:
    """
    Mengambil hasil analisis lengkap dari tabel, menyimpannya ke Excel,
    dan mengembalikan hasilnya sebagai DataFrame.

    Args:
        ticker_symbol (str): Simbol ticker yang akan dicari (e.g., 'BBCA.JK').

    Returns:
        pd.DataFrame: DataFrame yang berisi hasil analisis.
                      Mengembalikan DataFrame kosong jika tidak ada data atau error.
    """
    ticker = ticker_symbol.strip().upper()
    print(f"--- [Analisis] Mencari hasil analisis untuk '{ticker}' ---")

    try:
        conn_str = f"mysql+pymysql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}/{DB_CONFIG['database']}"
        engine = create_engine(conn_str)

        query = f"SELECT * FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %(ticker)s"
        df_result = pd.read_sql(query, engine, params={"ticker": ticker})

        if df_result.empty:
            print(f"Tidak ada hasil analisis yang ditemukan untuk ticker '{ticker}'.")
        else:
            print(f"Ditemukan {len(df_result)} baris hasil analisis.")
            filename = f"hasil_analisis_{ticker}.xlsx"
            df_result.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil analisis berhasil disimpan ke file: '{filename}'")

        return df_result

    except Exception as e:
        print(f"ERROR saat mengambil hasil analisis: {e}")
        traceback.print_exc()
        return pd.DataFrame()

In [2]:
data_hasil_analisis = get_analysis_results("BBCA.JK")

--- [Analisis] Mencari hasil analisis untuk 'BBCA.JK' ---
Ditemukan 2 baris hasil analisis.
Hasil analisis berhasil disimpan ke file: 'hasil_analisis_BBCA.JK.xlsx'


# NEWW

In [3]:
import pandas as pd
import pymysql
import json
import traceback

# ==============================================================================
# KONFIGURASI DATABASE - MOHON SESUAIKAN BAGIAN INI
# ==============================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel yang akan digunakan
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# ==============================================================================


def get_decomposed_analysis_data(ticker_symbol: str) -> dict:
    """
    Mengambil data analisis untuk satu ticker, kemudian membongkar kolom-kolom JSON
    spesifik menjadi DataFrame terpisah.

    Args:
        ticker_symbol (str): Simbol ticker yang akan dicari (e.g., 'BBCA.JK').

    Returns:
        dict: Sebuah dictionary di mana keys adalah nama-nama kolom dan
              values adalah DataFrame yang telah di-parse dari JSON.
              Mengembalikan dictionary kosong jika ticker tidak ditemukan atau terjadi error.
    """
    ticker = ticker_symbol.strip().upper()
    print(f"--- [Decompose] Memproses data analisis untuk '{ticker}' ---")

    # Kolom-kolom yang ingin kita proses sebagai JSON dan ubah menjadi DataFrame
    json_columns_to_process = [
        'all_individual_performance_top_10_combination_performance',
        'top_10_diff_ind_vs_ind',
        'top_10_diff_mixed_combo',
        'backtesting_insights',
        # Anda bisa menambahkan kolom lain di sini jika perlu
        'raw_data_with_signals',
        'individual_backtests',
        'combination_backtests',
        'signal_pairs_profit',
        'signal_counting',
    ]

    dataframes = {}
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        # Ambil semua kolom untuk ticker yang spesifik
        sql = f"SELECT * FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker,))
        result_row = cursor.fetchone()

        if not result_row:
            print(f"ERROR: Tidak ada data ditemukan untuk ticker '{ticker}' di tabel '{ANALYSIS_RESULTS_TABLE}'")
            return {}

        print(f"Data untuk '{ticker}' ditemukan. Memulai proses parsing kolom JSON...")

        # Loop melalui setiap kolom yang perlu kita proses
        for col_name in json_columns_to_process:
            print(f"  - Memproses kolom: '{col_name}'...")
            
            json_string = result_row.get(col_name)

            # Cek jika kolom ada dan tidak kosong
            if not json_string:
                print(f"    - WARNING: Kolom '{col_name}' kosong atau tidak ada. Membuat DataFrame kosong.")
                dataframes[col_name] = pd.DataFrame()
                continue
            
            # Coba parse JSON menjadi DataFrame
            try:
                # json.loads akan mengubah string JSON menjadi list of dictionaries
                parsed_data = json.loads(json_string)
                # pd.DataFrame akan mengubah list of dictionaries menjadi tabel
                df = pd.DataFrame(parsed_data)
                dataframes[col_name] = df
                print(f"    - SUKSES: Kolom '{col_name}' berhasil diubah menjadi DataFrame dengan {len(df)} baris.")
            except json.JSONDecodeError:
                print(f"    - ERROR: Gagal mem-parsing JSON di kolom '{col_name}'. Data mungkin corrupt.")
                print(f"      Data awal: {str(json_string)[:100]}...") # Tampilkan 100 karakter pertama dari data yang error
                dataframes[col_name] = pd.DataFrame() # Buat DataFrame kosong sebagai gantinya
            except Exception as e:
                print(f"    - ERROR: Terjadi kesalahan tak terduga saat memproses kolom '{col_name}': {e}")
                dataframes[col_name] = pd.DataFrame()

        return dataframes

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return {}
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return {}
    finally:
        if connection:
            connection.close()
            print("\nKoneksi ke database ditutup.")

In [5]:
all_dataframes = get_decomposed_analysis_data("AKRA.JK")

--- [Decompose] Memproses data analisis untuk 'AKRA.JK' ---
Data untuk 'AKRA.JK' ditemukan. Memulai proses parsing kolom JSON...
  - Memproses kolom: 'all_individual_performance_top_10_combination_performance'...
    - WARNING: Kolom 'all_individual_performance_top_10_combination_performance' kosong atau tidak ada. Membuat DataFrame kosong.
  - Memproses kolom: 'top_10_diff_ind_vs_ind'...
    - SUKSES: Kolom 'top_10_diff_ind_vs_ind' berhasil diubah menjadi DataFrame dengan 10 baris.
  - Memproses kolom: 'top_10_diff_mixed_combo'...
    - SUKSES: Kolom 'top_10_diff_mixed_combo' berhasil diubah menjadi DataFrame dengan 10 baris.
  - Memproses kolom: 'backtesting_insights'...
    - SUKSES: Kolom 'backtesting_insights' berhasil diubah menjadi DataFrame dengan 4 baris.
  - Memproses kolom: 'raw_data_with_signals'...
    - SUKSES: Kolom 'raw_data_with_signals' berhasil diubah menjadi DataFrame dengan 237 baris.
  - Memproses kolom: 'individual_backtests'...
    - SUKSES: Kolom 'individual_ba

In [7]:
if all_dataframes:
    df_performance = all_dataframes.get('all_individual_performance_top_10_combination_performance')
    df_diff_ind = all_dataframes.get('top_10_diff_ind_vs_ind')
    df_diff_mixed = all_dataframes.get('top_10_diff_mixed_combo')
    df_insights = all_dataframes.get('backtesting_insights')
    
    # Sekarang Anda bisa bekerja dengan df_performance, df_diff_ind, dll.
    if df_performance is not None:
        print(df_performance.describe())

ValueError: Cannot describe a DataFrame without columns

In [6]:
all_dataframes

{'all_individual_performance_top_10_combination_performance': Empty DataFrame
 Columns: []
 Index: [],
 'top_10_diff_ind_vs_ind':     ticker                                           strategy  \
 0  AKRA.JK  AKRA.JK - MACD_Signal (Buy) + Candlestick_Sign...   
 1  AKRA.JK  AKRA.JK - MACD_Signal (Buy) + Bollinger_Signal...   
 2  AKRA.JK  AKRA.JK - Candlestick_Signal (Buy) + Bollinger...   
 3  AKRA.JK  AKRA.JK - ADX_Signal (Buy) + Bollinger_Signal ...   
 4  AKRA.JK  AKRA.JK - ADX_Signal (Buy) + Candlestick_Signa...   
 5  AKRA.JK     AKRA.JK - ADX_Signal (Buy) + RSI_Signal (Sell)   
 6  AKRA.JK  AKRA.JK - Candlestick_Signal (Buy) + RSI_Signa...   
 7  AKRA.JK  AKRA.JK - MACD_Signal (Buy) + Fibonacci_Signal...   
 8  AKRA.JK  AKRA.JK - ADX_Signal (Buy) + Fibonacci_Signal ...   
 9  AKRA.JK  AKRA.JK - Candlestick_Signal (Buy) + Fibonacci...   
 
         buy_indicator      sell_indicator  initial_capital  final_capital  \
 0         MACD_Signal  Candlestick_Signal          1000000  9948

# NEWWWWWWWW

In [9]:
import pandas as pd
import pymysql
import json
import traceback
import os  # Modul untuk berinteraksi dengan sistem operasi (membuat folder)

# ==============================================================================
# KONFIGURASI DATABASE - MOHON SESUAIKAN BAGIAN INI
# ==============================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel yang akan digunakan
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# ==============================================================================


def get_decomposed_analysis_data(ticker_symbol: str) -> dict:
    """
    Mengambil data analisis untuk satu ticker, kemudian membongkar kolom-kolom JSON
    spesifik menjadi DataFrame terpisah. (Fungsi ini tidak berubah)
    """
    ticker = ticker_symbol.strip().upper()
    print(f"--- [Decompose] Memproses data analisis untuk '{ticker}' ---")

    json_columns_to_process = [
        'all_individual_performance_top_10_combination_performance',
        'top_10_diff_ind_vs_ind',
        'top_10_diff_mixed_combo',
        'backtesting_insights',
        'raw_data_with_signals',
        'individual_backtests',
        'combination_backtests',
        'signal_pairs_profit',
        'signal_counting',
    ]

    dataframes = {}
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        sql = f"SELECT * FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker,))
        result_row = cursor.fetchone()

        if not result_row:
            print(f"ERROR: Tidak ada data ditemukan untuk ticker '{ticker}'")
            return {}

        print(f"Data untuk '{ticker}' ditemukan. Memulai proses parsing kolom JSON...")
        for col_name in json_columns_to_process:
            print(f"  - Memproses kolom: '{col_name}'...")
            json_string = result_row.get(col_name)

            if not json_string:
                print(f"    - WARNING: Kolom '{col_name}' kosong. DataFrame tidak akan dibuat.")
                dataframes[col_name] = pd.DataFrame()
                continue
            
            try:
                parsed_data = json.loads(json_string)
                df = pd.DataFrame(parsed_data)
                dataframes[col_name] = df
                print(f"    - SUKSES: Kolom '{col_name}' berhasil diubah menjadi DataFrame dengan {len(df)} baris.")
            except Exception as e:
                print(f"    - ERROR: Terjadi kesalahan saat memproses kolom '{col_name}': {e}")
                dataframes[col_name] = pd.DataFrame()

        return dataframes

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return {}
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return {}
    finally:
        if connection:
            connection.close()
            print("\nKoneksi ke database ditutup.")


# --- FUNGSI BARU UNTUK MENYIMPAN HASIL ---
def save_dataframes_to_excel_folder(dataframes_dict: dict, ticker_symbol: str, folder_name: str = "hasil"):
    """
    Menyimpan setiap DataFrame dari dictionary ke file Excel terpisah di dalam folder tertentu.

    Args:
        dataframes_dict (dict): Dictionary berisi DataFrame yang akan disimpan.
        ticker_symbol (str): Simbol ticker, digunakan untuk penamaan file.
        folder_name (str): Nama folder tujuan untuk menyimpan file Excel.
    """
    if not dataframes_dict:
        print("Tidak ada DataFrame untuk disimpan.")
        return

    # 1. Buat folder tujuan jika belum ada
    try:
        os.makedirs(folder_name, exist_ok=True)
        print(f"\n--- Menyimpan file ke folder '{folder_name}' ---")
    except OSError as e:
        print(f"ERROR: Tidak dapat membuat folder '{folder_name}': {e}")
        return

    # 2. Loop melalui dictionary dan simpan setiap DataFrame
    for column_name, df in dataframes_dict.items():
        # Hanya simpan jika DataFrame tidak kosong
        if not df.empty:
            # 3. Buat nama file yang dinamis
            filename = f"{ticker_symbol}_{column_name}.xlsx"
            # os.path.join adalah cara yang aman untuk menggabungkan path folder dan nama file
            file_path = os.path.join(folder_name, filename)

            try:
                # 4. Simpan ke Excel tanpa kolom index dari pandas
                df.to_excel(file_path, index=False, engine='openpyxl')
                print(f"  -> Berhasil disimpan: {file_path}")
            except Exception as e:
                print(f"  -> GAGAL menyimpan {file_path}: {e}")
        else:
            print(f"  - Dilewati (kosong): {column_name}")

In [10]:
get_decomposed_analysis_data("AKRA.JK")

--- [Decompose] Memproses data analisis untuk 'AKRA.JK' ---
Data untuk 'AKRA.JK' ditemukan. Memulai proses parsing kolom JSON...
  - Memproses kolom: 'all_individual_performance_top_10_combination_performance'...
    - WARNING: Kolom 'all_individual_performance_top_10_combination_performance' kosong. DataFrame tidak akan dibuat.
  - Memproses kolom: 'top_10_diff_ind_vs_ind'...
    - SUKSES: Kolom 'top_10_diff_ind_vs_ind' berhasil diubah menjadi DataFrame dengan 10 baris.
  - Memproses kolom: 'top_10_diff_mixed_combo'...
    - SUKSES: Kolom 'top_10_diff_mixed_combo' berhasil diubah menjadi DataFrame dengan 10 baris.
  - Memproses kolom: 'backtesting_insights'...
    - SUKSES: Kolom 'backtesting_insights' berhasil diubah menjadi DataFrame dengan 4 baris.
  - Memproses kolom: 'raw_data_with_signals'...
    - SUKSES: Kolom 'raw_data_with_signals' berhasil diubah menjadi DataFrame dengan 237 baris.
  - Memproses kolom: 'individual_backtests'...
    - SUKSES: Kolom 'individual_backtests' ber

{'all_individual_performance_top_10_combination_performance': Empty DataFrame
 Columns: []
 Index: [],
 'top_10_diff_ind_vs_ind':     ticker                                           strategy  \
 0  AKRA.JK  AKRA.JK - MACD_Signal (Buy) + Candlestick_Sign...   
 1  AKRA.JK  AKRA.JK - MACD_Signal (Buy) + Bollinger_Signal...   
 2  AKRA.JK  AKRA.JK - Candlestick_Signal (Buy) + Bollinger...   
 3  AKRA.JK  AKRA.JK - ADX_Signal (Buy) + Bollinger_Signal ...   
 4  AKRA.JK  AKRA.JK - ADX_Signal (Buy) + Candlestick_Signa...   
 5  AKRA.JK     AKRA.JK - ADX_Signal (Buy) + RSI_Signal (Sell)   
 6  AKRA.JK  AKRA.JK - Candlestick_Signal (Buy) + RSI_Signa...   
 7  AKRA.JK  AKRA.JK - MACD_Signal (Buy) + Fibonacci_Signal...   
 8  AKRA.JK  AKRA.JK - ADX_Signal (Buy) + Fibonacci_Signal ...   
 9  AKRA.JK  AKRA.JK - Candlestick_Signal (Buy) + Fibonacci...   
 
         buy_indicator      sell_indicator  initial_capital  final_capital  \
 0         MACD_Signal  Candlestick_Signal          1000000  9948

In [11]:
save_dataframes_to_excel_folder(all_dataframes, "AKRA.JK", "hasil")


--- Menyimpan file ke folder 'hasil' ---
  - Dilewati (kosong): all_individual_performance_top_10_combination_performance
  -> Berhasil disimpan: hasil\AKRA.JK_top_10_diff_ind_vs_ind.xlsx
  -> Berhasil disimpan: hasil\AKRA.JK_top_10_diff_mixed_combo.xlsx
  -> Berhasil disimpan: hasil\AKRA.JK_backtesting_insights.xlsx
  -> Berhasil disimpan: hasil\AKRA.JK_raw_data_with_signals.xlsx
  -> Berhasil disimpan: hasil\AKRA.JK_individual_backtests.xlsx
  -> Berhasil disimpan: hasil\AKRA.JK_combination_backtests.xlsx
  -> Berhasil disimpan: hasil\AKRA.JK_signal_pairs_profit.xlsx
  -> Berhasil disimpan: hasil\AKRA.JK_signal_counting.xlsx


# GET TICKER SIGNAL

In [15]:
import pandas as pd
import pymysql
import json
import traceback

# ==============================================================================
# KONFIGURASI DATABASE - MOHON SESUAIKAN BAGIAN INI
# ==============================================================================
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel yang berisi hasil analisis
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# ==============================================================================


def get_ticker_signal(ticker_symbol: str) -> pd.DataFrame:
    """
    Mengambil data jumlah sinyal (Buy, Sell, Hold) untuk satu ticker spesifik
    dan mengubahnya menjadi DataFrame dengan format seperti gambar.

    Args:
        ticker_symbol (str): Simbol ticker yang akan dicari (e.g., 'BBNI.JK').

    Returns:
        pd.DataFrame: DataFrame satu baris yang berisi ringkasan sinyal untuk ticker tersebut.
                      Mengembalikan DataFrame kosong jika terjadi error atau data tidak ditemukan.
    """
    ticker = ticker_symbol.strip().upper()
    print(f"--- Mengambil ringkasan sinyal untuk ticker: {ticker} ---")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        # Ambil hanya kolom 'signal_counting' untuk efisiensi
        sql = f"SELECT `signal_counting` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker,))
        result = cursor.fetchone()

        if not result or not result.get('signal_counting'):
            print(f"ERROR: Tidak ada data 'signal_counting' yang ditemukan untuk ticker '{ticker}'")
            return pd.DataFrame()

        # Parse data JSON
        try:
            signal_data = json.loads(result['signal_counting'])
        except json.JSONDecodeError:
            print(f"ERROR: Gagal mem-parsing JSON untuk ticker '{ticker}'.")
            return pd.DataFrame()

        # --- Transformasi Data ke Format Tabel ---
        
        # Peta dari nama di JSON ke nama di header tabel (sesuaikan jika perlu)
        indicator_map = {
            'Bollinger': 'BOLLINGER BANDS',
            'MA': 'MA',
            'MACD': 'MACD',
            'RSI': 'RSI',
            'ADX': 'ADX',
            'Fibonacci': 'FIBONACCI',
            'Candlestick': 'CANDLESTICK' # Menambahkan dari contoh JSON Anda
        }
        
        # Urutan header sesuai gambar
        header_order = ['BOLLINGER BANDS', 'MA', 'MACD', 'RSI', 'ADX', 'FIBONACCI', 'CANDLESTICK']
        signal_types = ['Buy', 'Sell', 'Hold']
        
        flat_data = {}
        
        # Loop untuk mengisi data sesuai struktur header bertingkat
        for header_name in header_order:
            # Cari key asli di JSON yang sesuai dengan header
            json_key = next((key for key, val in indicator_map.items() if val == header_name), None)
            
            if json_key:
                for signal in signal_types:
                    # Kunci untuk DataFrame (tuple untuk MultiIndex)
                    column_key = (header_name, signal)
                    # Ambil data dengan aman, beri nilai 0 jika tidak ada
                    value = signal_data.get(json_key, {}).get(signal, 0)
                    flat_data[column_key] = value
        
        # Membuat DataFrame dari dictionary yang sudah diratakan
        df = pd.DataFrame([flat_data])
        
        # Menambahkan kolom Ticker di posisi awal
        df.insert(0, 'Ticker', ticker)
        
        print(f"Berhasil membuat ringkasan untuk {ticker}.")
        return df

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()


def get_all_ticker_signals(save_to_excel: bool = True) -> pd.DataFrame:
    """
    Mengambil data jumlah sinyal untuk SEMUA ticker yang ada di database,
    menggabungkannya menjadi satu DataFrame, dan menyimpannya ke Excel.

    Args:
        save_to_excel (bool): Jika True, akan menyimpan hasilnya ke file Excel.

    Returns:
        pd.DataFrame: DataFrame gabungan yang berisi ringkasan sinyal untuk semua ticker.
    """
    print("\n=== Memulai proses untuk mengambil ringkasan sinyal semua ticker ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG)
        cursor = connection.cursor()

        # 1. Dapatkan daftar semua ticker unik
        sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
        cursor.execute(sql)
        tickers = [row[0] for row in cursor.fetchall()]
        
        if not tickers:
            print("Tidak ada ticker yang ditemukan di database.")
            return pd.DataFrame()
            
        print(f"Ditemukan {len(tickers)} ticker unik. Memproses satu per satu...")

        # 2. Loop melalui setiap ticker dan kumpulkan hasilnya
        all_dfs = []
        for ticker in tickers:
            df_single = get_ticker_signal(ticker)
            if not df_single.empty:
                all_dfs.append(df_single)

        if not all_dfs:
            print("Tidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        # 3. Gabungkan semua DataFrame menjadi satu
        final_df = pd.concat(all_dfs, ignore_index=True)

        # 4. Buat kolom 'No'
        final_df.insert(0, 'No', range(1, len(final_df) + 1))
        
        print(f"\nBerhasil menggabungkan data untuk {len(final_df)} ticker.")

        # 5. Simpan ke Excel jika diminta
        if save_to_excel:
            filename = "ringkasan_sinyal_semua_ticker.xlsx"
            final_df.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")
            
        return final_df

    except pymysql.Error as e:
        print(f"DATABASE ERROR saat mengambil daftar ticker: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR saat memproses semua ticker: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [16]:
bbni_signal_summary = get_ticker_signal('BBNI.JK')

--- Mengambil ringkasan sinyal untuk ticker: BBNI.JK ---
Berhasil membuat ringkasan untuk BBNI.JK.


In [17]:
all_signals_summary = get_all_ticker_signals(save_to_excel=True)


=== Memulai proses untuk mengambil ringkasan sinyal semua ticker ===
Ditemukan 47 ticker unik. Memproses satu per satu...
--- Mengambil ringkasan sinyal untuk ticker: ABDA.JK ---
Berhasil membuat ringkasan untuk ABDA.JK.
--- Mengambil ringkasan sinyal untuk ticker: ACES.JK ---
Berhasil membuat ringkasan untuk ACES.JK.
--- Mengambil ringkasan sinyal untuk ticker: ADES.JK ---
Berhasil membuat ringkasan untuk ADES.JK.
--- Mengambil ringkasan sinyal untuk ticker: ADMR.JK ---
Berhasil membuat ringkasan untuk ADMR.JK.
--- Mengambil ringkasan sinyal untuk ticker: ADRO.JK ---
Berhasil membuat ringkasan untuk ADRO.JK.
--- Mengambil ringkasan sinyal untuk ticker: AKRA.JK ---
Berhasil membuat ringkasan untuk AKRA.JK.
--- Mengambil ringkasan sinyal untuk ticker: AMMN.JK ---
Berhasil membuat ringkasan untuk AMMN.JK.
--- Mengambil ringkasan sinyal untuk ticker: AMRT.JK ---
Berhasil membuat ringkasan untuk AMRT.JK.
--- Mengambil ringkasan sinyal untuk ticker: ANTM.JK ---
Berhasil membuat ringkasan u

# INDIVIDUAL PAIRS BACKTEST

In [18]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel yang berisi hasil analisis
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# ==============================================================================


def get_indicator_performance_summary(ticker_symbol: str) -> pd.DataFrame:
    """
    Mengambil data detail transaksi dari 'signal_pairs_profit', menghitung metrik
    kinerja untuk setiap indikator, dan menyajikannya dalam format tabel ringkasan.

    Args:
        ticker_symbol (str): Simbol ticker yang akan dicari (e.g., 'ACES').

    Returns:
        pd.DataFrame: DataFrame yang berisi ringkasan kinerja per indikator.
                      Mengembalikan DataFrame kosong jika terjadi error atau data tidak ditemukan.
    """
    ticker = ticker_symbol.strip().upper()
    print(f"--- Menghitung ringkasan kinerja indikator untuk: {ticker} ---")

    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        sql = f"SELECT `signal_pairs_profit` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker,))
        result = cursor.fetchone()

        if not result or not result.get('signal_pairs_profit'):
            print(f"ERROR: Tidak ada data 'signal_pairs_profit' yang ditemukan untuk ticker '{ticker}'")
            return pd.DataFrame()

        try:
            full_data = json.loads(result['signal_pairs_profit'])
            # Fokus pada data 'individual'
            individual_data = full_data.get('individual')
            if not individual_data:
                print(f"ERROR: Tidak ada 'individual' section di dalam JSON untuk ticker '{ticker}'")
                return pd.DataFrame()
        except (json.JSONDecodeError, AttributeError):
            print(f"ERROR: Gagal mem-parsing JSON atau struktur data tidak valid untuk ticker '{ticker}'.")
            return pd.DataFrame()

        # --- Proses Perhitungan untuk Setiap Indikator ---
        summary_rows = []
        
        # Peta untuk membuat nama indikator lebih rapi
        indicator_name_map = {
            "Bollinger": "Bollinger Bands",
            "MA": "Moving Average (MA)",
            "RSI": "RSI",
            "MACD": "MACD",
            "ADX": "ADX",
            "Fibonacci": "Fibonacci",
            "Candlestick": "Candlestick Pattern"
        }

        for indicator_key, trades in individual_data.items():
            # Lewati jika tidak ada transaksi untuk indikator ini
            if not trades:
                continue

            num_trades = len(trades)
            
            # Hitung jumlah trade yang untung (profit > 0)
            winning_trades = sum(1 for trade in trades if trade.get('profit_pct', 0) > 0)
            
            # Hitung Win Rate, hindari pembagian dengan nol
            win_rate = (winning_trades / num_trades) * 100 if num_trades > 0 else 0
            
            # Hitung rata-rata profit dan holding period
            total_profit_pct = sum(trade.get('profit_pct', 0) for trade in trades)
            avg_profit_pct = total_profit_pct / num_trades if num_trades > 0 else 0
            
            total_holding_days = sum(trade.get('holding_days', 0) for trade in trades)
            avg_holding_days = total_holding_days / num_trades if num_trades > 0 else 0
            
            # Buat baris data untuk DataFrame
            row_data = {
                'Ticker': ticker,
                'Indikator': indicator_name_map.get(indicator_key, indicator_key),
                'Total Profit (%)': avg_profit_pct,
                'Holding Rata-rata (Hari)': int(round(avg_holding_days, 0)),
                'Jumlah transaksi': num_trades,
                'Win Rate': win_rate
            }
            summary_rows.append(row_data)

        if not summary_rows:
            print(f"Tidak ada data transaksi yang valid untuk diproses pada ticker {ticker}.")
            return pd.DataFrame()

        # Buat DataFrame dari hasil kalkulasi
        df = pd.DataFrame(summary_rows)

        # --- Formatting Akhir ---
        
        # Atur urutan kolom sesuai gambar
        column_order = ['Ticker', 'Indikator', 'Total Profit (%)', 'Holding Rata-rata (Hari)', 'Jumlah transaksi', 'Win Rate']
        df = df[column_order]

        # Format kolom persentase agar mudah dibaca
        df['Total Profit (%)'] = df['Total Profit (%)'].map('{:.2f}%'.format)
        df['Win Rate'] = df['Win Rate'].map('{:.2f}%'.format)
        
        # Tambahkan kolom 'No' di awal
        df.insert(0, 'No', range(1, len(df) + 1))
        
        print(f"Berhasil membuat ringkasan kinerja untuk {ticker}.")
        return df

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [22]:
performance_summary = get_indicator_performance_summary('BBCA.JK')

--- Menghitung ringkasan kinerja indikator untuk: BBCA.JK ---
Berhasil membuat ringkasan kinerja untuk BBCA.JK.


In [23]:
performance_summary

,No,Ticker,Indikator,Total Profit (%),Holding Rata-rata (Hari),Jumlah transaksi,Win Rate
0,1,BBCA.JK,Bollinger Bands,2.23%,32,1,100.00%
1,2,BBCA.JK,Moving Average (MA),-1.47%,91,1,0.00%
2,3,BBCA.JK,RSI,-10.51%,166,1,0.00%
3,4,BBCA.JK,MACD,-2.83%,10,14,7.14%
4,5,BBCA.JK,ADX,0.99%,48,4,50.00%
5,6,BBCA.JK,Fibonacci,-12.67%,175,1,0.00%
6,7,BBCA.JK,Candlestick Pattern,-0.62%,6,31,32.26%


## all ticker

In [24]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel yang berisi hasil analisis
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# ==============================================================================


def get_indicator_performance_summary(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal) Menghitung metrik kinerja untuk setiap indikator untuk SATU ticker.
    Fungsi ini sekarang menerima objek koneksi untuk efisiensi.
    """
    ticker = ticker_symbol.strip().upper()
    
    try:
        cursor = connection.cursor()
        sql = f"SELECT `signal_pairs_profit` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker,))
        result = cursor.fetchone()

        if not result or not result.get('signal_pairs_profit'):
            print(f"  - WARNING: Tidak ada data 'signal_pairs_profit' untuk {ticker}.")
            return pd.DataFrame()

        individual_data = json.loads(result['signal_pairs_profit']).get('individual')
        if not individual_data:
            return pd.DataFrame()

        summary_rows = []
        indicator_name_map = {
            "Bollinger": "Bollinger Bands", "MA": "Moving Average (MA)", "RSI": "RSI",
            "MACD": "MACD", "ADX": "ADX", "Fibonacci": "Fibonacci", "Candlestick": "Candlestick Pattern"
        }

        for indicator_key, trades in individual_data.items():
            if not trades: continue
            num_trades = len(trades)
            winning_trades = sum(1 for trade in trades if trade.get('profit_pct', 0) > 0)
            win_rate = (winning_trades / num_trades) * 100 if num_trades > 0 else 0
            avg_profit_pct = sum(t.get('profit_pct', 0) for t in trades) / num_trades if num_trades > 0 else 0
            avg_holding_days = sum(t.get('holding_days', 0) for t in trades) / num_trades if num_trades > 0 else 0
            
            summary_rows.append({
                'Ticker': ticker,
                'Indikator': indicator_name_map.get(indicator_key, indicator_key),
                'Total Profit (%)': f"{avg_profit_pct:.2f}%",
                'Holding Rata-rata (Hari)': int(round(avg_holding_days)),
                'Jumlah transaksi': num_trades,
                'Win Rate': f"{win_rate:.2f}%"
            })

        return pd.DataFrame(summary_rows)
    except Exception as e:
        print(f"  - ERROR saat memproses {ticker}: {e}")
        return pd.DataFrame()


# --- FUNGSI BARU UNTUK PROSES BANYAK TICKER ---
def get_bulk_indicator_performance(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Menjalankan proses ringkasan kinerja indikator untuk banyak ticker sekaligus,
    menggabungkan hasilnya, dan menyimpannya ke Excel.

    Args:
        tickers (list, optional): Daftar ticker yang akan diproses.
                                  Jika None, akan memproses SEMUA ticker di database. Defaults to None.
        save_to_excel (bool, optional): Jika True, menyimpan hasil ke file Excel. Defaults to True.

    Returns:
        pd.DataFrame: DataFrame gabungan yang berisi ringkasan kinerja.
    """
    print("\n=== Memulai proses ringkasan kinerja untuk banyak ticker ===")
    
    connection = None
    try:
        # Buat satu koneksi untuk digunakan oleh semua proses
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        # Jika daftar ticker tidak diberikan, ambil semua dari DB
        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan di database.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        # Proses setiap ticker dan kumpulkan hasilnya
        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = get_indicator_performance_summary(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        # Gabungkan semua DataFrame menjadi satu
        final_df = pd.concat(all_dfs, ignore_index=True)
        
        # Atur ulang urutan kolom
        column_order = ['Ticker', 'Indikator', 'Total Profit (%)', 'Holding Rata-rata (Hari)', 'Jumlah transaksi', 'Win Rate']
        final_df = final_df[column_order]

        # Tambahkan kolom 'No' di awal setelah semua data digabung
        final_df.insert(0, 'No', range(1, len(final_df) + 1))

        print(f"\nBerhasil menggabungkan data. Total {len(final_df)} baris dibuat.")

        # Simpan ke file Excel jika diminta
        if save_to_excel:
            filename = "ringkasan_kinerja_indikator_semua_ticker.xlsx"
            final_df.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return final_df

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [25]:
all_ticker_summary = get_bulk_indicator_performance()


=== Memulai proses ringkasan kinerja untuk banyak ticker ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.JK...
(31/

## newww

In [50]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# ==============================================================================


def get_indicator_performance_summary(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal Helper) Menghitung metrik kinerja untuk setiap indikator untuk SATU ticker.
    (Tidak ada perubahan di sini)
    """
    ticker = ticker_symbol.strip().upper()
    
    try:
        cursor = connection.cursor()
        sql = f"SELECT `signal_pairs_profit` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker,))
        result = cursor.fetchone()

        if not result or not result.get('signal_pairs_profit'):
            # Pesan ini sekarang lebih baik ditempatkan di sini
            # print(f"  - WARNING: Tidak ada data 'signal_pairs_profit' untuk {ticker}.")
            return pd.DataFrame()

        individual_data = json.loads(result['signal_pairs_profit']).get('individual')
        if not individual_data:
            return pd.DataFrame()

        summary_rows = []
        indicator_name_map = {
            "Bollinger": "Bollinger Bands", "MA": "Moving Average (MA)", "RSI": "RSI",
            "MACD": "MACD", "ADX": "ADX", "Fibonacci": "Fibonacci", "Candlestick": "Candlestick Pattern"
        }

        for indicator_key, trades in individual_data.items():
            if not trades: continue
            num_trades = len(trades)
            winning_trades = sum(1 for trade in trades if trade.get('profit_pct', 0) > 0)
            win_rate = (winning_trades / num_trades) * 100 if num_trades > 0 else 0
            avg_profit_pct = sum(t.get('profit_pct', 0) for t in trades) / num_trades if num_trades > 0 else 0
            avg_holding_days = sum(t.get('holding_days', 0) for t in trades) / num_trades if num_trades > 0 else 0
            
            summary_rows.append({
                'Ticker': ticker,
                'Indikator': indicator_name_map.get(indicator_key, indicator_key),
                'Total Profit (%)': f"{avg_profit_pct:.2f}%",
                'Holding Rata-rata (Hari)': int(round(avg_holding_days)),
                'Jumlah transaksi': num_trades,
                'Win Rate': f"{win_rate:.2f}%"
            })

        return pd.DataFrame(summary_rows)
    except Exception as e:
        print(f"  - ERROR saat memproses {ticker}: {e}")
        return pd.DataFrame()


# ==============================================================================
# FUNGSI YANG DIPERBAIKI
# ==============================================================================
def get_bulk_indicator_performance(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Menjalankan proses ringkasan kinerja indikator untuk banyak ticker,
    MENGURUTKANNYA BERDASARKAN INDIKATOR, dan menyimpannya ke Excel. (VERSI DIPERBAIKI)
    """
    print("\n=== Memulai proses ringkasan kinerja untuk banyak ticker ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = get_indicator_performance_summary(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        final_df = pd.concat(all_dfs, ignore_index=True)
        
        column_order = ['Ticker', 'Indikator', 'Total Profit (%)', 'Holding Rata-rata (Hari)', 'Jumlah transaksi', 'Win Rate']
        final_df = final_df[column_order]

        # --- PERUBAHAN UTAMA: PENGURUTAN DATA ---
        # Mengurutkan DataFrame berdasarkan 'Indikator' (utama) lalu 'Ticker' (sekunder).
        print("Mengurutkan data berdasarkan Indikator, lalu Ticker...")
        final_df = final_df.sort_values(by=['Indikator', 'Ticker']).reset_index(drop=True)
        # ---------------------------------------------

        # Tambahkan kolom 'No' di awal SETELAH diurutkan
        final_df.insert(0, 'No', range(1, len(final_df) + 1))

        print(f"\nBerhasil menggabungkan dan mengurutkan data. Total {len(final_df)} baris dibuat.")

        if save_to_excel:
            filename = "neww_ringkasan_kinerja_indikator_diurutkan_per_indikator.xlsx"
            final_df.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return final_df

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [51]:
sorted_performance = get_bulk_indicator_performance()


=== Memulai proses ringkasan kinerja untuk banyak ticker ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.JK...
(31/

# SUMMARY PAIR SIGNAL BACKTEST

In [28]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel yang berisi hasil analisis
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# ==============================================================================


def _create_summary_for_ticker(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal Helper) Membuat tabel ringkasan 3 baris (Individual, Kombinasi, Keseluruhan)
    untuk SATU ticker.
    """
    try:
        cursor = connection.cursor()
        sql = f"SELECT `signal_pairs_profit` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker_symbol,))
        result = cursor.fetchone()

        if not result or not result.get('signal_pairs_profit'):
            return pd.DataFrame()

        # Ambil data summary dari JSON
        parsed_json = json.loads(result['signal_pairs_profit'])
        summary_data = parsed_json.get('summary')
        if not summary_data:
            return pd.DataFrame()

        summary_rows = []
        
        # Definisikan kategori dan kunci JSON yang sesuai
        categories = {
            'Individual': 'individual_stats',
            'Kombinasi': 'combination_stats',
            'Keseluruhan': 'overall_stats'
        }

        for category_name, stats_key in categories.items():
            stats = summary_data.get(stats_key, {})
            
            # Buat satu baris data
            row = {
                'Ticker': ticker_symbol,
                'Kategori Backtesting': category_name,
                'Total Skenario': stats.get('count', 0),
                'Total Transaksi': stats.get('total_trades', 0),
                'Rata-rata Profit': stats.get('avg_profit', 0),
                'Indikator Terbaik': stats.get('best_indicator', 'N/A'),
                'Profit Tertinggi': stats.get('best_profit', 0),
                'Indikator Terburuk': stats.get('worst_indicator', 'N/A'),
                'Profit Terburuk': stats.get('worst_profit', 0),
                'Rata-rata Winrate': stats.get('avg_win_rate', 0)
            }
            summary_rows.append(row)
        
        return pd.DataFrame(summary_rows)

    except (json.JSONDecodeError, KeyError) as e:
        print(f"  - ERROR: Gagal memproses JSON untuk {ticker_symbol}: {e}")
        return pd.DataFrame()


def get_backtesting_summary_bulk(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Membuat tabel ringkasan backtesting untuk banyak ticker, menggabungkannya,
    dan memformatnya agar sesuai dengan gambar yang diberikan.

    Args:
        tickers (list, optional): Daftar ticker. Jika None, semua ticker akan diproses.
        save_to_excel (bool, optional): Jika True, simpan ke file Excel.

    Returns:
        pd.DataFrame: DataFrame ringkasan gabungan.
    """
    print("\n=== Memulai proses pembuatan Ringkasan Backtesting Gabungan ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = _create_summary_for_ticker(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        # Gabungkan semua DataFrame
        final_df = pd.concat(all_dfs, ignore_index=True)

        # --- Formatting agar persis seperti gambar ---

        # 1. Format kolom persentase
        for col in ['Rata-rata Profit', 'Profit Tertinggi', 'Profit Terburuk', 'Rata-rata Winrate']:
            final_df[col] = final_df[col].map('{:.2f}%'.format)

        # 2. Tambahkan kolom 'No' dan kosongkan duplikat
        # Buat nomor unik untuk setiap ticker
        final_df.insert(0, 'No', final_df.groupby('Ticker').ngroup() + 1)
        # Kosongkan nomor untuk baris kedua dan ketiga dari ticker yang sama
        final_df['No'] = final_df['No'].astype(str).mask(final_df.duplicated('Ticker'), '')
        
        # 3. Kosongkan nama Ticker untuk baris kedua dan ketiga
        final_df['Ticker'] = final_df['Ticker'].mask(final_df.duplicated('Ticker'), '')
        
        print(f"\nBerhasil menggabungkan data. Total {len(final_df)} baris dibuat.")

        if save_to_excel:
            filename = "pair_signal_summary.xlsx"
            final_df.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return final_df

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [29]:
summary_table = get_backtesting_summary_bulk()


=== Memulai proses pembuatan Ringkasan Backtesting Gabungan ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.JK...
(

# BACKTESTING INDIVIDU

In [33]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel yang berisi hasil analisis
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# Kolom sumber data untuk ringkasan ini
SOURCE_COLUMN = 'all_individual_performance'
# ==============================================================================


def _create_detailed_summary_for_ticker(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal Helper) Membuat tabel detail dari setiap skenario backtest individual
    untuk SATU ticker.
    """
    try:
        cursor = connection.cursor()
        # Menggunakan backticks untuk nama kolom yang panjang
        sql = f"SELECT `{SOURCE_COLUMN}` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker_symbol,))
        result = cursor.fetchone()

        if not result or not result.get(SOURCE_COLUMN):
            return pd.DataFrame()

        parsed_data = json.loads(result[SOURCE_COLUMN])
        if not isinstance(parsed_data, list):
            return pd.DataFrame()

        # Langsung ubah list of dictionaries menjadi DataFrame
        df = pd.DataFrame(parsed_data)
        
        # Jika DataFrame kosong setelah parsing, keluar
        if df.empty:
            return pd.DataFrame()

        return df

    except (json.JSONDecodeError, KeyError) as e:
        print(f"  - ERROR: Gagal memproses JSON untuk {ticker_symbol}: {e}")
        return pd.DataFrame()


def get_detailed_backtest_summary(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Membuat tabel ringkasan detail dari setiap skenario backtest individual untuk
    banyak ticker, menggabungkannya, dan memformatnya.

    Args:
        tickers (list, optional): Daftar ticker. Jika None, semua ticker akan diproses.
        save_to_excel (bool, optional): Jika True, simpan ke file Excel.

    Returns:
        pd.DataFrame: DataFrame ringkasan detail gabungan.
    """
    print("\n=== Memulai proses pembuatan Ringkasan Detail Backtest Individual ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = _create_detailed_summary_for_ticker(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        # Gabungkan semua DataFrame
        final_df = pd.concat(all_dfs, ignore_index=True)

        # --- Mapping, Seleksi, dan Formatting Kolom ---

        # 1. Peta dari nama kolom di JSON ke nama kolom di tabel akhir
        column_map = {
            'ticker': 'Ticker',
            'indicator': 'Indikator',
            'initial_capital': 'Modal Awal',
            'final_capital': 'Modal Akhir',
            'total_profit': 'Jumlah Profit',
            'profit_percentage': 'Total Profit (%)',
            'num_executed_cycles': 'Jumlah Cycle',
            'num_trades': 'Jumlah transaksi',
            'win_rate': 'Win Rate',
            'volume_analysis_summary': 'Volume Analisis'
        }
        
        # 2. Pilih hanya kolom yang kita butuhkan dan ganti namanya
        df_formatted = final_df[list(column_map.keys())].rename(columns=column_map)

        # 3. Bersihkan nama indikator
        df_formatted['Indikator'] = df_formatted['Indikator'].str.replace('_Signal', '', regex=False).str.replace('_', ' ')

        # 4. Format kolom numerik dan persentase
        for col in ['Modal Awal', 'Modal Akhir', 'Jumlah Profit']:
            df_formatted[col] = df_formatted[col].apply(lambda x: int(round(x, 0)))
        
        for col in ['Total Profit (%)', 'Win Rate']:
             df_formatted[col] = df_formatted[col].map('{:.2f}%'.format)

        # 5. Tambahkan kolom 'No' di awal
        df_formatted.insert(0, 'No', range(1, len(df_formatted) + 1))
        
        print(f"\nBerhasil menggabungkan data. Total {len(df_formatted)} baris dibuat.")

        if save_to_excel:
            filename = "backtesting_individu.xlsx"
            df_formatted.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return df_formatted

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [34]:
detailed_summary_table = get_detailed_backtest_summary()


=== Memulai proses pembuatan Ringkasan Detail Backtest Individual ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.J

## neww

In [48]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# Pastikan nama kolom ini benar sesuai dengan database Anda
# Berdasarkan kode Anda, sumbernya adalah 'all_individual_performance'
SOURCE_COLUMN = 'all_individual_performance'
# ==============================================================================


def _create_detailed_summary_for_ticker(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal Helper) Membuat tabel detail dari setiap skenario backtest individual
    untuk SATU ticker. (Tidak ada perubahan di sini)
    """
    try:
        cursor = connection.cursor()
        sql = f"SELECT `{SOURCE_COLUMN}` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker_symbol,))
        result = cursor.fetchone()

        if not result or not result.get(SOURCE_COLUMN):
            return pd.DataFrame()

        parsed_data = json.loads(result[SOURCE_COLUMN])
        if not isinstance(parsed_data, list):
            return pd.DataFrame()

        df = pd.DataFrame(parsed_data)
        
        if df.empty:
            return pd.DataFrame()

        return df

    except (json.JSONDecodeError, KeyError) as e:
        print(f"  - ERROR: Gagal memproses JSON untuk {ticker_symbol}: {e}")
        return pd.DataFrame()


# ==============================================================================
# FUNGSI YANG DIPERBAIKI
# ==============================================================================
def get_detailed_backtest_summary(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Membuat tabel ringkasan detail dari setiap skenario backtest individual untuk
    banyak ticker, MENGURUTKANNYA BERDASARKAN INDIKATOR, dan memformatnya. (VERSI DIPERBAIKI)
    """
    print("\n=== Memulai proses pembuatan Ringkasan Detail Backtest Individual ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = _create_detailed_summary_for_ticker(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        final_df = pd.concat(all_dfs, ignore_index=True)

        # --- Mapping, Seleksi, dan Formatting Kolom ---

        column_map = {
            'ticker': 'Ticker',
            'indicator': 'Indikator',
            'initial_capital': 'Modal Awal',
            'final_capital': 'Modal Akhir',
            'total_profit': 'Jumlah Profit',
            'profit_percentage': 'Total Profit (%)',
            'num_executed_cycles': 'Jumlah Cycle',
            'num_trades': 'Jumlah transaksi',
            'win_rate': 'Win Rate',
            'volume_analysis_summary': 'Volume Analisis'
        }
        
        # Cek kolom yang hilang untuk mencegah KeyError
        for col_key in column_map.keys():
            if col_key not in final_df.columns:
                final_df[col_key] = pd.NA

        df_formatted = final_df[list(column_map.keys())].rename(columns=column_map)

        df_formatted['Indikator'] = df_formatted['Indikator'].str.replace('_Signal', '', regex=False).str.replace('_', ' ')

        # --- PERUBAHAN UTAMA: PENGURUTAN DATA ---
        # Mengurutkan DataFrame berdasarkan 'Indikator' (utama) lalu 'Ticker' (sekunder).
        print("Mengurutkan data berdasarkan Indikator, lalu Ticker...")
        df_formatted = df_formatted.sort_values(by=['Indikator', 'Ticker']).reset_index(drop=True)
        # .reset_index(drop=True) penting untuk membuat ulang index dari 0 setelah pengurutan,
        # agar penomoran berikutnya menjadi benar.
        # ---------------------------------------------
        
        # Format kolom numerik dengan penanganan NaN
        for col in ['Modal Awal', 'Modal Akhir', 'Jumlah Profit']:
            # Menggunakan .fillna(0) untuk mengatasi error jika data hilang
            df_formatted[col] = df_formatted[col].fillna(0).astype(int)
        
        for col in ['Total Profit (%)', 'Win Rate']:
             df_formatted[col] = df_formatted[col].fillna(0).map('{:.2f}%'.format)

        # Tambahkan kolom 'No' di awal SETELAH diurutkan
        df_formatted.insert(0, 'No', range(1, len(df_formatted) + 1))
        
        print(f"\nBerhasil menggabungkan dan mengurutkan data. Total {len(df_formatted)} baris dibuat.")

        if save_to_excel:
            filename = "neww_backtesting_individu_diurutkan_per_indikator.xlsx"
            df_formatted.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return df_formatted

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [49]:
sorted_summary = get_detailed_backtest_summary()


=== Memulai proses pembuatan Ringkasan Detail Backtest Individual ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.J

# TOP KOMBINASI 1 INDIKATOR

In [35]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel dan kolom sumber
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
# Anda menyebutkan top_10_combination_performance, kita akan gunakan itu.
# Pastikan nama kolom ini sesuai dengan yang ada di database Anda.
SOURCE_COLUMN_COMBINATION = 'top_10_combination_performance'
# ==============================================================================


def _create_top_combo_summary_for_ticker(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal Helper) Mengambil data untuk KOMBINASI TERBAIK (data pertama)
    untuk SATU ticker.
    """
    try:
        cursor = connection.cursor()
        sql = f"SELECT `{SOURCE_COLUMN_COMBINATION}` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker_symbol,))
        result = cursor.fetchone()

        if not result or not result.get(SOURCE_COLUMN_COMBINATION):
            return pd.DataFrame()

        parsed_data = json.loads(result[SOURCE_COLUMN_COMBINATION])
        
        # Logika Kunci: Ambil hanya data pertama jika ini adalah list
        if isinstance(parsed_data, list) and parsed_data:
            top_1_combo_data = parsed_data[0] # Ambil elemen pertama
            # Ubah menjadi DataFrame satu baris
            return pd.DataFrame([top_1_combo_data])
        else:
            return pd.DataFrame()

    except (json.JSONDecodeError, IndexError, KeyError) as e:
        print(f"  - ERROR: Gagal memproses JSON/data untuk {ticker_symbol}: {e}")
        return pd.DataFrame()


def get_top_combination_summary(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Membuat tabel ringkasan dari skenario backtest kombinasi terbaik (Top 1) untuk
    banyak ticker, menggabungkannya, dan memformatnya.

    Args:
        tickers (list, optional): Daftar ticker. Jika None, semua ticker akan diproses.
        save_to_excel (bool, optional): Jika True, simpan ke file Excel.

    Returns:
        pd.DataFrame: DataFrame ringkasan detail gabungan.
    """
    print("\n=== Memulai proses pembuatan Ringkasan Kombinasi Indikator Terbaik ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = _create_top_combo_summary_for_ticker(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        # Gabungkan semua DataFrame
        final_df = pd.concat(all_dfs, ignore_index=True)

        # --- Mapping, Seleksi, dan Formatting Kolom ---

        column_map = {
            'ticker': 'Ticker',
            'indicator': 'Indikator Kombinasi Terbaik',
            'initial_capital': 'Modal Awal',
            'final_capital': 'Modal Akhir',
            'total_profit': 'Jumlah Profit',
            'profit_percentage': 'Total Profit (%)',
            'num_executed_cycles': 'Jumlah Cycle',
            'num_trades': 'Jumlah transaksi',
            'win_rate': 'Win Rate',
            'volume_analysis_summary': 'Volume Analisis'
        }
        
        # Pilih hanya kolom yang kita butuhkan dan ganti namanya
        df_formatted = final_df[list(column_map.keys())].rename(columns=column_map)

        # Bersihkan nama indikator
        df_formatted['Indikator Kombinasi Terbaik'] = df_formatted['Indikator Kombinasi Terbaik'].str.replace('_Combined_Signal', '', regex=False)

        # Format kolom numerik dan persentase
        for col in ['Modal Awal', 'Modal Akhir', 'Jumlah Profit']:
            df_formatted[col] = df_formatted[col].apply(lambda x: int(round(x, 0)))
        
        for col in ['Total Profit (%)', 'Win Rate']:
             df_formatted[col] = df_formatted[col].map('{:.2f}%'.format)

        # Tambahkan kolom 'No' di awal
        df_formatted.insert(0, 'No', range(1, len(df_formatted) + 1))
        
        print(f"\nBerhasil menggabungkan data. Total {len(df_formatted)} baris dibuat.")

        if save_to_excel:
            filename = "individu_ringkasan_kombinasi_terbaik.xlsx"
            df_formatted.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return df_formatted

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [36]:
top_combination_table = get_top_combination_summary()


=== Memulai proses pembuatan Ringkasan Kombinasi Indikator Terbaik ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.

# TOP INDIVIDU KOMBINASI BEDA INDIKATOR

In [42]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel dan kolom sumber
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
SOURCE_COLUMN_PAIRS = 'top_10_diff_ind_vs_ind'
# ==============================================================================


def _create_top_pair_summary_for_ticker(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal Helper) Mengambil data untuk PASANGAN INDIKATOR TERBAIK (data pertama)
    untuk SATU ticker.
    """
    try:
        cursor = connection.cursor()
        sql = f"SELECT `{SOURCE_COLUMN_PAIRS}` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker_symbol,))
        result = cursor.fetchone()

        if not result or not result.get(SOURCE_COLUMN_PAIRS):
            return pd.DataFrame()

        parsed_data = json.loads(result[SOURCE_COLUMN_PAIRS])
        
        if isinstance(parsed_data, list) and parsed_data:
            top_1_pair_data = parsed_data[0]  # Ambil elemen pertama

            # Tambahkan nilai default jika tidak ada
            if 'initial_capital' not in top_1_pair_data:
                top_1_pair_data['initial_capital'] = 1_000_000

            return pd.DataFrame([top_1_pair_data])
        else:
            return pd.DataFrame()

    except (json.JSONDecodeError, IndexError, KeyError) as e:
        print(f"  - ERROR: Gagal memproses JSON/data untuk {ticker_symbol}: {e}")
        return pd.DataFrame()


def get_top_indicator_pair_summary(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Membuat tabel ringkasan dari skenario backtest pasangan indikator terbaik (Top 1)
    untuk banyak ticker, menggabungkannya, dan memformatnya.

    Args:
        tickers (list, optional): Daftar ticker. Jika None, semua ticker akan diproses.
        save_to_excel (bool, optional): Jika True, simpan ke file Excel.

    Returns:
        pd.DataFrame: DataFrame ringkasan detail gabungan.
    """
    print("\n=== Memulai proses pembuatan Ringkasan Pasangan Indikator Terbaik ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = _create_top_pair_summary_for_ticker(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        final_df = pd.concat(all_dfs, ignore_index=True)

        # --- Mapping, Seleksi, dan Formatting Kolom ---

        column_map = {
            'ticker': 'Ticker',
            'buy_indicator': 'Buy Indikator Terbaik',
            'sell_indicator': 'Sell Indikator Terbaik',
            'initial_capital': 'Modal Awal',
            'final_capital': 'Modal Akhir',
            'total_profit': 'Jumlah Profit',
            'return_percentage': 'Total Profit (%)',
            'num_executed_cycles': 'Jumlah Cycle',
            'avg_holding_period': 'Rata-Rata Holding',
            'total_trades': 'Jumlah transaksi',
            'win_rate': 'Win Rate',
            'volume_analysis_summary': 'Volume Analysis'
        }
        
        df_formatted = final_df[list(column_map.keys())].rename(columns=column_map)

        # Bersihkan nama indikator
        for col in ['Buy Indikator Terbaik', 'Sell Indikator Terbaik']:
            df_formatted[col] = df_formatted[col].str.replace('_Signal', '', regex=False).str.replace('_', ' ')

        # Format kolom numerik dan persentase
        for col in ['Modal Awal', 'Modal Akhir', 'Jumlah Profit']:
            df_formatted[col] = df_formatted[col].apply(lambda x: int(round(x, 0)))
        
        df_formatted['Rata-Rata Holding'] = df_formatted['Rata-Rata Holding'].map('{:.2f}'.format)
        
        for col in ['Total Profit (%)', 'Win Rate']:
             df_formatted[col] = df_formatted[col].map('{:.2f}%'.format)

        df_formatted.insert(0, 'No', range(1, len(df_formatted) + 1))
        
        print(f"\nBerhasil menggabungkan data. Total {len(df_formatted)} baris dibuat.")

        if save_to_excel:
            filename = "top_individu_gabungan_indikator.xlsx"
            df_formatted.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return df_formatted

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [43]:
top_pair_table = get_top_indicator_pair_summary()


=== Memulai proses pembuatan Ringkasan Pasangan Indikator Terbaik ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.J

# TOP KOMBINASI KOMBINASI INDIKATOR

In [44]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel dan kolom sumber
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
SOURCE_COLUMN_MIXED_COMBO = 'top_10_diff_mixed_combo'
# ==============================================================================


def _create_top_mixed_summary_for_ticker(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal Helper) Mengambil data untuk KOMBINASI CAMPURAN TERBAIK (data pertama)
    untuk SATU ticker.
    """
    try:
        cursor = connection.cursor()
        sql = f"SELECT `{SOURCE_COLUMN_MIXED_COMBO}` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker_symbol,))
        result = cursor.fetchone()

        if not result or not result.get(SOURCE_COLUMN_MIXED_COMBO):
            return pd.DataFrame()

        # Data di kolom ini adalah satu dictionary, bukan list.
        # Jadi kita langsung parsing.
        parsed_data = json.loads(result[SOURCE_COLUMN_MIXED_COMBO])
        
        # Logika Kunci: Ambil hanya data pertama jika ini adalah list
        if isinstance(parsed_data, list) and parsed_data:
            top_1_mixed_data = parsed_data[0] # Ambil elemen pertama dari list
            return pd.DataFrame([top_1_mixed_data]) # Ubah menjadi DataFrame satu baris
        else:
            return pd.DataFrame()

    except (json.JSONDecodeError, IndexError, KeyError) as e:
        print(f"  - ERROR: Gagal memproses JSON/data untuk {ticker_symbol}: {e}")
        return pd.DataFrame()


def get_top_mixed_combo_summary(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Membuat tabel ringkasan dari skenario backtest kombinasi campuran terbaik (Top 1)
    untuk banyak ticker, menggabungkannya, dan memformatnya.

    Args:
        tickers (list, optional): Daftar ticker. Jika None, semua ticker akan diproses.
        save_to_excel (bool, optional): Jika True, simpan ke file Excel.

    Returns:
        pd.DataFrame: DataFrame ringkasan detail gabungan.
    """
    print("\n=== Memulai proses pembuatan Ringkasan Kombinasi Campuran Terbaik ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = _create_top_mixed_summary_for_ticker(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        final_df = pd.concat(all_dfs, ignore_index=True)

        # --- Mapping, Seleksi, dan Formatting Kolom ---
        column_map = {
            'ticker': 'Ticker',
            'buy_indicator': 'Buy Indikator Terbaik',
            'sell_indicator': 'Sell Indikator Terbaik',
            'initial_capital': 'Modal Awal',
            'final_capital': 'Modal Akhir',
            'total_profit': 'Jumlah Profit',
            'return_percentage': 'Total Profit (%)',
            'num_executed_cycles': 'Jumlah Cycle',
            'avg_holding_period': 'Rata-Rata Holding',
            'total_trades': 'Jumlah transaksi',
            'win_rate': 'Win Rate',
            'volume_analysis_summary': 'Volume Analysis'
        }
        
        # Cek jika ada kolom yang hilang di DataFrame dan tambahkan sebagai NaN
        for col_key in column_map.keys():
            if col_key not in final_df.columns:
                final_df[col_key] = pd.NA

        df_formatted = final_df[list(column_map.keys())].rename(columns=column_map)

        # Bersihkan nama indikator
        for col in ['Buy Indikator Terbaik', 'Sell Indikator Terbaik']:
            df_formatted[col] = (df_formatted[col]
                                 .str.replace('_Signal', '', regex=False)
                                 .str.replace('_Combined', '', regex=False)
                                 .str.replace('_', ' '))

        # Format kolom numerik dengan penanganan NaN
        for col in ['Modal Awal', 'Modal Akhir', 'Jumlah Profit']:
            df_formatted[col] = df_formatted[col].fillna(0).astype(int)
        
        df_formatted['Rata-Rata Holding'] = df_formatted['Rata-Rata Holding'].fillna(0).map('{:.2f}'.format)
        
        for col in ['Total Profit (%)', 'Win Rate']:
             df_formatted[col] = df_formatted[col].fillna(0).map('{:.2f}%'.format)

        df_formatted.insert(0, 'No', range(1, len(df_formatted) + 1))
        
        print(f"\nBerhasil menggabungkan data. Total {len(df_formatted)} baris dibuat.")

        if save_to_excel:
            filename = "top_kombinasi_kombinasi indikator.xlsx"
            df_formatted.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return df_formatted

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [45]:
top_mixed_combo_table = get_top_mixed_combo_summary()


=== Memulai proses pembuatan Ringkasan Kombinasi Campuran Terbaik ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.J

# BACKTESTING INSIGHT

In [46]:
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel dan kolom sumber
ANALYSIS_RESULTS_TABLE = 'hasil_analisis_lengkap'
SOURCE_COLUMN_INSIGHTS = 'backtesting_insights'
# ==============================================================================


def _create_insights_summary_for_ticker(ticker_symbol: str, connection) -> pd.DataFrame:
    """
    (Internal Helper) Membuat tabel ringkasan dari 'backtesting_insights' untuk SATU ticker.
    """
    try:
        cursor = connection.cursor()
        sql = f"SELECT `{SOURCE_COLUMN_INSIGHTS}` FROM `{ANALYSIS_RESULTS_TABLE}` WHERE `ticker` = %s"
        cursor.execute(sql, (ticker_symbol,))
        result = cursor.fetchone()

        if not result or not result.get(SOURCE_COLUMN_INSIGHTS):
            return pd.DataFrame()

        parsed_data = json.loads(result[SOURCE_COLUMN_INSIGHTS])
        
        # Langsung ubah list of dictionaries menjadi DataFrame
        df = pd.DataFrame(parsed_data)
        
        if df.empty:
            return pd.DataFrame()
            
        # Tambahkan kolom ticker untuk proses grouping nanti
        df['ticker'] = ticker_symbol
        return df

    except (json.JSONDecodeError, KeyError) as e:
        print(f"  - ERROR: Gagal memproses JSON untuk {ticker_symbol}: {e}")
        return pd.DataFrame()


def get_backtesting_insights_summary(tickers: list = None, save_to_excel: bool = True) -> pd.DataFrame:
    """
    Membuat tabel ringkasan dari backtesting insights untuk banyak ticker,
    menggabungkannya, dan memformatnya agar sesuai dengan gambar.

    Args:
        tickers (list, optional): Daftar ticker. Jika None, semua ticker akan diproses.
        save_to_excel (bool, optional): Jika True, simpan ke file Excel.

    Returns:
        pd.DataFrame: DataFrame ringkasan gabungan.
    """
    print("\n=== Memulai proses pembuatan Ringkasan Backtesting Insights ===")
    
    connection = None
    try:
        connection = pymysql.connect(**DB_CONFIG, cursorclass=pymysql.cursors.DictCursor)
        cursor = connection.cursor()

        if tickers is None:
            print("Mengambil daftar semua ticker dari database...")
            sql = f"SELECT DISTINCT `ticker` FROM `{ANALYSIS_RESULTS_TABLE}` ORDER BY `ticker`"
            cursor.execute(sql)
            tickers = [row['ticker'] for row in cursor.fetchall()]
            if not tickers:
                print("Tidak ada ticker yang ditemukan.")
                return pd.DataFrame()
            print(f"Ditemukan {len(tickers)} ticker unik.")

        all_dfs = []
        for i, ticker in enumerate(tickers, 1):
            print(f"({i}/{len(tickers)}) Memproses {ticker}...")
            df_single = _create_insights_summary_for_ticker(ticker, connection)
            if not df_single.empty:
                all_dfs.append(df_single)
        
        if not all_dfs:
            print("\nTidak ada data yang berhasil diproses.")
            return pd.DataFrame()

        final_df = pd.concat(all_dfs, ignore_index=True)

        # --- Mapping, Seleksi, dan Formatting Kolom ---

        # 1. Peta dari nama kategori di JSON ke nama yang diinginkan di tabel
        category_map = {
            'Individual': 'Individual',
            'Combination': 'Kombinasi',
            'Different_Combination': 'Kombinasi Berbeda'
            # Kita tidak memasukkan 'Overall' agar terfilter secara otomatis
        }
        
        # 2. Peta dari nama kolom di JSON ke nama kolom di tabel
        column_map = {
            'ticker': 'Ticker',
            'category': 'Kategori Backtesting',
            'total_strategies': 'Total Skenario',
            'avg_profit': 'Rata-rata Profit',
            'avg_profit_percentage': 'Rata-rata Persentase Profit',
            'avg_win_rate': 'Rata-rata Win Rate'
        }
        
        # Terapkan pemetaan kategori
        final_df['category'] = final_df['category'].map(category_map)
        # Hapus baris yang kategorinya tidak ada di peta (misal: 'Overall')
        final_df = final_df.dropna(subset=['category'])
        
        # Pilih kolom yang dibutuhkan dan ganti namanya
        df_formatted = final_df[list(column_map.keys())].rename(columns=column_map)

        # 3. Format kolom numerik dan persentase
        df_formatted['Rata-rata Profit'] = df_formatted['Rata-rata Profit'].fillna(0).astype(int)
        
        for col in ['Rata-rata Persentase Profit', 'Rata-rata Win Rate']:
             df_formatted[col] = df_formatted[col].fillna(0).map('{:.2f}%'.format)
        
        # 4. Format 'grouping' agar No dan Ticker hanya muncul sekali per grup
        df_formatted.insert(0, 'No', df_formatted.groupby('Ticker').ngroup() + 1)
        df_formatted['No'] = df_formatted['No'].astype(str).mask(df_formatted.duplicated('Ticker'), '')
        df_formatted['Ticker'] = df_formatted['Ticker'].mask(df_formatted.duplicated('Ticker'), '')

        print(f"\nBerhasil menggabungkan data. Total {len(df_formatted)} baris dibuat.")

        if save_to_excel:
            filename = "ringkasan_backtesting_insights.xlsx"
            df_formatted.to_excel(filename, index=False, engine='openpyxl')
            print(f"Hasil telah disimpan ke file: '{filename}'")

        return df_formatted

    except pymysql.Error as e:
        print(f"DATABASE ERROR: {e}")
        return pd.DataFrame()
    except Exception as e:
        print(f"GENERAL ERROR: {e}")
        traceback.print_exc()
        return pd.DataFrame()
    finally:
        if connection:
            connection.close()

In [47]:
insights_summary_table = get_backtesting_insights_summary()


=== Memulai proses pembuatan Ringkasan Backtesting Insights ===
Mengambil daftar semua ticker dari database...
Ditemukan 47 ticker unik.
(1/47) Memproses ABDA.JK...
(2/47) Memproses ACES.JK...
(3/47) Memproses ADES.JK...
(4/47) Memproses ADMR.JK...
(5/47) Memproses ADRO.JK...
(6/47) Memproses AKRA.JK...
(7/47) Memproses AMMN.JK...
(8/47) Memproses AMRT.JK...
(9/47) Memproses ANTM.JK...
(10/47) Memproses ARTO.JK...
(11/47) Memproses ASII.JK...
(12/47) Memproses BBCA.JK...
(13/47) Memproses BBNI.JK...
(14/47) Memproses BBRI.JK...
(15/47) Memproses BBTN.JK...
(16/47) Memproses BMRI.JK...
(17/47) Memproses BRIS.JK...
(18/47) Memproses BRPT.JK...
(19/47) Memproses CPIN.JK...
(20/47) Memproses CTRA.JK...
(21/47) Memproses ESSA.JK...
(22/47) Memproses EXCL.JK...
(23/47) Memproses GOTO.JK...
(24/47) Memproses ICBP.JK...
(25/47) Memproses INCO.JK...
(26/47) Memproses INDF.JK...
(27/47) Memproses INKP.JK...
(28/47) Memproses ISAT.JK...
(29/47) Memproses ITMG.JK...
(30/47) Memproses JPFA.JK...
(

# GET TREND DATA FULL

In [1]:
import pandas as pd
import pymysql
import traceback

# --- KONFIGURASI ---

# Konfigurasi koneksi ke database Anda
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'harga_saham'
}

# Nama tabel sumber di database
SOURCE_TABLE = 'dashboard_notification_trend_mei'

# Daftar ticker saham yang termasuk dalam indeks LQ45 (perkiraan, bisa disesuaikan)
# Pastikan formatnya sama dengan yang ada di kolom 'Ticker' database Anda (misal: 'BBCA.JK')
LQ45_TICKERS = [
    'ACES.JK', 'ADMR.JK', 'ADRO.JK', 'AKRA.JK', 'AMMN.JK', 'AMRT.JK', 'ANTM.JK',
    'ARTO.JK', 'ASII.JK', 'BBCA.JK', 'BBNI.JK', 'BBRI.JK', 'BBTN.JK', 'BMRI.JK',
    'BRIS.JK', 'BRPT.JK', 'CPIN.JK', 'CTRA.JK', 'ESSA.JK', 'EXCL.JK', 'GOTO.JK',
    'ICBP.JK', 'INCO.JK', 'INDF.JK', 'INKP.JK', 'ISAT.JK', 'ITMG.JK', 'JPFA.JK',
    'JSMR.JK', 'KLBF.JK', 'MAPA.JK', 'MAPI.JK', 'MBMA.JK', 'MDKA.JK', 'MEDC.JK',
    'PGAS.JK', 'PGEO.JK', 'PTBA.JK', 'SIDO.JK', 'SMGR.JK', 'SMRA.JK', 'TLKM.JK',
    'TOWR.JK', 'UNTR.JK', 'UNVR.JK'
]


def export_lq45_trend_to_excel(filename: str = "lq45_stock_trends.xlsx"):
    """
    Mengambil data tren dari database untuk saham-saham LQ45,
    memformatnya sesuai gambar, dan menyimpannya ke file Excel.

    Args:
        filename (str, optional): Nama file Excel output. 
                                  Defaultnya "lq45_stock_trends.xlsx".
    """
    print("\n=== Memulai proses ekspor data tren saham LQ45 ===")
    
    connection = None
    try:
        # 1. Menghubungkan ke database
        print(f"Menghubungkan ke database '{DB_CONFIG['database']}'...")
        connection = pymysql.connect(**DB_CONFIG)
        
        # 2. Menyiapkan query SQL untuk mengambil data LQ45
        # Membuat placeholder (%s) sebanyak jumlah ticker di list LQ45
        placeholders = ', '.join(['%s'] * len(LQ45_TICKERS))
        
        # Kolom yang ingin diambil, sesuaikan jika nama di DB berbeda
        # Urutan ini juga akan menjadi urutan di file Excel
        columns_to_select = [
            'Ticker', 'bollinger', 'ma', 'rsi', 'macd', 
            'adx', 'volume', 'candlestick', 'fibonacci', 'overall'
        ]
        columns_sql_string = ', '.join([f"`{col}`" for col in columns_to_select])

        sql_query = f"""
            SELECT {columns_sql_string}
            FROM `{SOURCE_TABLE}`
            WHERE `Ticker` IN ({placeholders})
            ORDER BY `Ticker` ASC
        """
        
        # 3. Mengambil data menggunakan Pandas
        print(f"Mengambil data untuk {len(LQ45_TICKERS)} saham LQ45...")
        # pd.read_sql_query sangat efisien untuk tugas ini
        df = pd.read_sql_query(sql_query, connection, params=LQ45_TICKERS)
        
        if df.empty:
            print("Tidak ada data yang ditemukan untuk ticker LQ45 di tabel tersebut.")
            return

        print(f"Berhasil menemukan data untuk {len(df)} saham.")

        # 4. Formatting DataFrame agar sesuai dengan gambar
        # Menambahkan kolom terakhir dengan isi "{...}" dan header kosong
        print("Memformat data untuk disesuaikan dengan output...")
        df[' '] = '{...}' # Nama kolom adalah spasi agar terlihat kosong di Excel

        # 5. Menyimpan DataFrame ke file Excel
        print(f"Menyimpan data ke file Excel: '{filename}'")
        df.to_excel(filename, index=False, engine='openpyxl')
        
        print("\n=== Proses Selesai! ===")
        print(f"File '{filename}' berhasil dibuat di direktori yang sama dengan script ini.")

    except pymysql.Error as e:
        print(f"DATABASE ERROR: Terjadi kesalahan saat berinteraksi dengan MySQL.")
        print(f"Detail Error: {e}")
        traceback.print_exc()
    except Exception as e:
        print(f"GENERAL ERROR: Terjadi kesalahan yang tidak terduga.")
        print(f"Detail Error: {e}")
        traceback.print_exc()
    finally:
        # Pastikan koneksi selalu ditutup
        if connection:
            connection.close()
            print("Koneksi ke database telah ditutup.")


In [2]:
export_lq45_trend_to_excel()


=== Memulai proses ekspor data tren saham LQ45 ===
Menghubungkan ke database 'harga_saham'...
Mengambil data untuk 45 saham LQ45...


C:\Users\proda\AppData\Local\Temp\ipykernel_27188\1805131481.py:70: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql_query, connection, params=LQ45_TICKERS)


Berhasil menemukan data untuk 45 saham.
Memformat data untuk disesuaikan dengan output...
Menyimpan data ke file Excel: 'lq45_stock_trends.xlsx'

=== Proses Selesai! ===
File 'lq45_stock_trends.xlsx' berhasil dibuat di direktori yang sama dengan script ini.
Koneksi ke database telah ditutup.
